# Group 3 - Final Project
### Evan S, Joey G, Ryan S, Jack V

## Introduction
The data we will be working on is a comprehensive dataset of NBA player, game, and season statistics from 1947-present. The dataset originates from kaggle, which was scraped from basketball-reference.com. Advanced statistics were not calculated until 1980, so we will filter for data between the years 1980 and 2024.

Link: https://www.kaggle.com/datasets/sumitrodatta/nba-aba-baa-stats?select=Advanced.csv

In [23]:
# Import data & clean
import numpy as np
import pandas as pd
# load in data
end_of_season_teams = pd.read_csv('nba_data/End of Season Teams (Voting).csv')
player_awards = pd.read_csv('nba_data/Player Award Shares.csv')
player_career = pd.read_csv('nba_data/Player Career Info.csv')
player_season = pd.read_csv('nba_data/Player Season Info.csv')
player_shooting = pd.read_csv('nba_data/Player Shooting.csv')
player_totals = pd.read_csv('nba_data/Player Totals.csv')
team_summary = pd.read_csv('nba_data/Team Summaries.csv')
# check to make sure it loaded in
#end_of_season_teams.head(50)
#player_awards.head()   # get rid of useless awards
#player_career.head(50)   # used for num_seasons
#player_season.head()   # only experience == 1 for roty data
player_shooting.head() #
#player_totals.head()   #
#team_summary.head()    #

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,...,fg_percent_from_x16_3p_range,fg_percent_from_x3p_range,percent_assisted_x2p_fg,percent_assisted_x3p_fg,percent_dunks_of_fga,num_of_dunks,percent_corner_3s_of_3pa,corner_3_point_percent,num_heaves_attempted,num_heaves_made
0,31871.0,2025,5025.0,A.J. Green,NaN,SG,25,3.0,NBA,MIL,...,0.435,0.414,0.731,0.942,0.000,0,0.312,0.519,2,1
1,31872.0,2025,5026.0,A.J. Lawson,NaN,SG,24,3.0,NBA,TOR,...,NaN,0.316,0.778,0.958,0.067,7,0.382,0.310,0,0
2,31873.0,2025,5210.0,AJ Johnson,NaN,SG,20,1.0,NBA,TOT,...,NaN,0.291,0.667,0.625,0.117,12,0.109,0.333,1,0
3,31874.0,2025,5210.0,AJ Johnson,NaN,SG,20,1.0,NBA,MIL,...,NaN,0.600,0.600,0.667,0.053,0,0.200,1.000,0,0
4,31875.0,2025,5210.0,AJ Johnson,NaN,SG,20,1.0,NBA,WAS,...,NaN,0.260,0.677,0.615,0.128,12,0.100,0.200,1,0


In [29]:
# cleaning end_of_season_teams
end_of_season_teams2 = end_of_season_teams[
                        (end_of_season_teams['type'] == 'All-NBA') &
                        (end_of_season_teams['number_tm'] == '1T')][
                            ['season', 'type', 'player', 'age', 'tm', 'pts_won','seas_id']]
#end_of_season_teams2.head()


# cleaning player_awards
dpoy = player_awards[(player_awards['award'] == 'dpoy') & 
                        (player_awards['winner'] == True)][
                            ['season', 'award', 'player', 'age', 'tm', 'pts_won','seas_id']]
mvp = player_awards[(player_awards['award'] == 'nba mvp') & 
                        (player_awards['winner'] == True)][
                            ['season', 'award', 'player', 'age', 'tm', 'pts_won','seas_id']]
#dpoy.head(10)
#mvp.head(10)

# cleaning player_awards
dpoy = player_awards[(player_awards['award'] == 'dpoy') & 
                        (player_awards['winner'] == True)][
                            ['season', 'award', 'player', 'age', 'tm', 'pts_won','seas_id']]

# cleaning player_career
player_career = player_career[['player_id', 'player', 'num_seasons']]
#player_career.head(10)


# cleaning player_season
player_season = player_season[['season', 'seas_id', 'player', 'pos', 'age', 'tm', 'experience']]
#player_season.head(10)

# cleaning player_shooting
season_team_count = player_shooting.groupby(['player', 'season'])['tm'].nunique()
players_single_team = season_team_count[season_team_count == 1].to_frame()
players_single_team.head(20)
# only keeping players who were on a single team during a season
#player_shooting = 

tm
player        season    
A.C. Green    1998     1
              1999     1
              2000     1
              2001     1
A.J. Bramlett 2000     1
A.J. Green    2023     1
              2024     1
              2025     1
A.J. Guyton   2001     1
              2002     1
              2003     1
A.J. Hammons  2017     1
A.J. Lawson   2024     1
              2025     1
A.J. Price    2010     1
              2011     1
              2012     1
              2013     1
              2014     1
AJ Griffin    2023     1

## Data Summary

In [ ]:
# Feature descriptions

## Motivating Questions

## EDA

In [ ]:
# EDA

## Model Building & Implementation

In [ ]:
# Models

## Conclusions

## Limitations & Recommendations for Future Work